In [1]:
import os
import sys
import pandas as pd

sys.path.insert(0, r'C:\\Users\\Matt\\Documents\\Development\\VolAdjGIPDashboard\\VolAdjGIPDashboard')
#sys.path.insert(0, '/home/matt/src/VolAdjGIPDashboard/VolAdjGIPDashboard')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "VolAdjGIPDashboard.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

import django
django.setup()

In [2]:
from DataAcquisition.models import QuadForecasts
gdp, cpi, latest_date = QuadForecasts.fetch_usa_gi_data() # 20191108, both series verified against FRED.
display(gdp.tail(5))
display(cpi.tail(5))

DATE
2019-09-30    19141.744
2019-12-31    19253.959
2020-03-31    19010.848
2020-06-30    17302.511
2020-09-30    18583.501
Freq: Q-DEC, Name: GDPC1, dtype: float64

2020-06-30 00:00:00    256.860000
2020-09-30 00:00:00    259.766333
2020-12-31             260.095000
2021-03-31             262.463333
2021-06-30             264.373333
dtype: float64

In [3]:
gdp_set, latest_date = QuadForecasts.get_gdp_set(gdp) # issue is probably related to GDPNow. # maybe not; deleted code, still lying to me

In [4]:
gdp_set.tail(20)

growth  best_estimate
quarter    date                               
2020-09-30 2020-10-02  1.033390   17880.243800
           2020-10-09  1.033458   17881.419603
           2020-10-16  1.032823   17870.436407
           2020-10-23  1.032733   17868.865725
           2020-10-30  1.032369   17862.578850
2020-12-31 2020-09-04  1.017700   18912.420068
           2020-09-11  1.017723   18912.860820
           2020-09-18  1.013043   18825.878348
           2020-09-25  1.012393   18813.801153
           2020-10-02  1.011717   18801.252038
           2020-10-09  1.011838   18803.494795
           2020-10-16  1.008759   18746.278346
           2020-10-23  1.008540   18742.203708
           2020-10-30  1.008003   18732.232277
           2020-11-06  1.007075   18714.971297
           2020-11-13  1.007075   18714.971297
           2020-11-20  1.007075   18714.971297
           2020-11-27  1.006977   18713.151571
           2020-12-04  1.006241   18699.486675
2021-03-31 2020-12-04  1.014339   18967.612000

In [5]:
for forecast in QuadForecasts.objects.all().order_by(('date')):
    print(f"quarter_end_date={forecast.quarter_end_date}, date={forecast.date}, quad={forecast.quad}")

quarter_end_date=1951-06-30, date=1951-06-30, quad=3
quarter_end_date=1951-09-30, date=1951-09-30, quad=4
quarter_end_date=1951-12-31, date=1951-12-31, quad=4
quarter_end_date=1952-03-31, date=1952-03-31, quad=4
quarter_end_date=1952-06-30, date=1952-06-30, quad=4
quarter_end_date=1952-09-30, date=1952-09-30, quad=3
quarter_end_date=1952-12-31, date=1952-12-31, quad=1
quarter_end_date=1953-03-31, date=1953-03-31, quad=1
quarter_end_date=1953-06-30, date=1953-06-30, quad=2
quarter_end_date=1953-09-30, date=1953-09-30, quad=4
quarter_end_date=1953-12-31, date=1953-12-31, quad=3
quarter_end_date=1954-03-31, date=1954-03-31, quad=3
quarter_end_date=1954-06-30, date=1954-06-30, quad=4
quarter_end_date=1954-09-30, date=1954-09-30, quad=1
quarter_end_date=1954-12-31, date=1954-12-31, quad=1
quarter_end_date=1955-03-31, date=1955-03-31, quad=1
quarter_end_date=1955-06-30, date=1955-06-30, quad=2
quarter_end_date=1955-09-30, date=1955-09-30, quad=2
quarter_end_date=1955-12-31, date=1955-12-31, 

In [7]:
gdp, cpi, latest_data = QuadForecasts.fetch_usa_gi_data()
usa_quads = QuadForecasts.determine_quads(gdp, cpi).dropna()

usa_quads = usa_quads[
    (usa_quads.index.get_level_values('date') <= usa_quads.index.get_level_values('quarter')) &
    (usa_quads.index.get_level_values('date') > usa_quads.index.get_level_values('quarter') - pd.offsets.QuarterEnd())
].droplevel('quarter')

usa_quads['changed'] = usa_quads.quad != usa_quads.shift(1).quad
#usa_quads = usa_quads[usa_quads.changed == True].drop(['changed'], axis='columns')

AttributeError: 'tuple' object has no attribute 'dropna'

In [6]:
usa_quads.tail(20)

,gdp_roc,cpi_roc,quad,changed
date,,,,
2019-08-30,-20.498506,-5.388707,4.0,False
2019-09-06,-20.498506,-5.388707,4.0,False
2019-09-13,-20.498506,-5.388707,4.0,False
2019-09-20,-20.498506,-5.388707,4.0,False
2019-09-27,-20.498506,-5.388707,4.0,False
2019-10-04,5.303102,33.255639,2.0,True
2019-10-11,5.303102,33.255639,2.0,False
2019-10-18,-1.003723,33.255639,3.0,True
2019-10-25,-4.287898,33.255639,3.0,False
